In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# prepare right and left striatal masks

mask = nib.load('/content/drive/MyDrive/1 Sagol Project/Mask/Tian_Subcortex_S3_3T.nii').get_fdata()

striatum = list(range(11, 19)) + list(range(22,24)) + list(range(36, 44)) + list(range(47,49))

bool_mask = np.isin(mask, striatum)

mask[~bool_mask] = 0
mask[bool_mask] = 1


r_mask, l_mask = np.copy(mask), np.copy(mask)
r_mask[44:,:,:] = 0;
r_mask = r_mask.flatten()
l_mask[:44,:,:] = 0;
l_mask = l_mask.flatten()
mask = mask.flatten()

In [8]:
# load PET files and separate right and left striatal PET scans using masks

pet_folder_path = '/content/drive/MyDrive/1 Sagol Project/PET/'

original_Y_right, original_Y_left = [], []

for file in os.listdir(pet_folder_path):
  file_path = os.path.join(pet_folder_path, file)
  temp_pet = nib.load(file_path).get_fdata()
  temp_pet = temp_pet.flatten()
  original_Y_right.append(temp_pet[r_mask != 0])
  original_Y_left.append(temp_pet[l_mask != 0])



In [9]:
# select only subjects which have clinical lateralization scores

Y_right, Y_left = [], []

for i in range(26):
  if i not in (5, 15, 17, 20, 22):
    Y_right.append(original_Y_right[i])
    Y_left.append(original_Y_left[i])

In [10]:
# calculate mean PET activation in right and left striatum

mean_SUVR_right = [Y_right[i].mean() for i in range(21)]
mean_SUVR_left = [Y_left[i].mean() for i in range(21)]

In [11]:
# assign lateralization label based on PET
# 0 - right, 1 - left

pet_labels = [(mean_SUVR_right[i] > mean_SUVR_left[i]).astype(int) for i in range(21)]

In [12]:
# lateralization label based on clinical scores
# 0 - right, 1 - left

clinical_labels = [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

In [13]:
# check for which subjects the labels are matching

comp = [clinical_labels[i] == pet_labels[i] for i in range(21)]
comp

[True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True]